# An example of a microservice in Adama

This example can be run by checking out the repository

> https://github.com/waltermoreira/adama-example.git

and using `docker-compose up`.  See the [README](https://github.com/waltermoreira/adama-example/blob/master/README) file for more details.